In [1]:
import numpy as np
import re
from datetime import datetime, timedelta
import uuid

# User

In [2]:
class User:
    def __init__(self, user_id, interested_topics):
        self._user_id = user_id
        self._interested_topics = interested_topics
        self._bookings = []

    # Getter and Setter for user_id
    @property
    def user_id(self):
        return self._user_id

    @user_id.setter
    def user_id(self, user_id):
        self._user_id = user_id

    # Getter and Setter for interested_topics
    @property
    def interested_topics(self):
        return self._interested_topics

    @interested_topics.setter
    def interested_topics(self, interested_topics):
        self._interested_topics = interested_topics

    # Getter and Setter for bookings
    @property
    def bookings(self):
        return self._bookings

    @bookings.setter
    def bookings(self, value):
        if isinstance(value, list):
            self._bookings = value
        else:
            raise ValueError("Bookings must be a list")
        
    def add_booking(self, booking):
        self._bookings.append(booking)

    def remove_booking(self, booking_id):
        """
        Remove a booking by its ID.
        """
        booking_to_remove = next((b for b in self._bookings if b.booking_id == booking_id), None)
        if booking_to_remove:
            self._bookings.remove(booking_to_remove)
        else:
            print(f"Booking with ID {booking_id} not found in user's bookings.")

    def has_overlapping_conference(self, start_timestamp, end_timestamp, conferences):
        """
        Checks if the user has any overlapping conference bookings with the given start and end timestamps.
        """
        for booking in self._bookings:
            conference = conferences[booking.conference_name]
            if (start_timestamp < conference.end_timestamp and end_timestamp > conference.start_timestamp):
                return True
        return False

In [3]:
class UserSystem:
    def __init__(self):
        self.users = {}

    def add_user(self, user_id, interested_topics):
        """
        Add User: Add a new user to the system. It takes following parameters

        UserID: An Alphanumeric string with no special characters allowed
        Interested Topics: Comma Separated Strings. Strings can be Alphanumeric with spaces allowed. Maximum 50 Strings

        Expectations: UserID would be unique in the system. No endpoint to edit an existing user is needed.
        """
        # Validate user_id
        if not re.match(r'^[a-zA-Z0-9]+$', user_id):
            raise ValueError("Invalid user_id. Only alphanumeric characters are allowed.")

        # Validate interested topics
        topics_list = [topic.strip() for topic in interested_topics.split(",")]
        if len(topics_list) > 50 or any(not re.match(r'^[a-zA-Z0-9 ]+$', topic) for topic in topics_list):
            raise ValueError("Invalid interested topics. Must be comma-separated strings with alphanumeric characters and spaces, maximum 50 strings.")

        # Check for unique user_id
        if user_id in self.users:
            raise ValueError("UserID must be unique.")

        # Add the user
        new_user = User(user_id, interested_topics)
        self.users[user_id] = new_user

        return new_user
    
    def user_exists(self, user_id):
        """
        Check if a user with the given name exists in the system.
        """
        return user_id in self.users
    
    def get_user(self, user_id):
        """
        Return a user.
        """
        return self.users[user_id]

# Booking

In [4]:
class Booking:
    def __init__(self, booking_id, conference_name, user_id, status='waiting'):
        self._booking_id = booking_id
        self._conference_name = conference_name
        self._user_id = user_id
        self._status = status

    # Getter and Setter for booking_id
    @property
    def booking_id(self):
        return self._booking_id

    @booking_id.setter
    def booking_id(self, value):
        self._booking_id = value

    # Getter and Setter for conference_name
    @property
    def conference_name(self):
        return self._conference_name

    @conference_name.setter
    def conference_name(self, value):
        self._conference_name = value

    # Getter and Setter for user_id
    @property
    def user_id(self):
        return self._user_id

    @user_id.setter
    def user_id(self, value):
        self._user_id = value

    # Getter and Setter for status
    @property
    def status(self):
        return self._status

    @status.setter
    def status(self, value):
        if value in ['accepted', 'cancelled', 'waiting']:
            self._status = value
        else:
            raise ValueError("Invalid status. Must be 'accepted', 'cancelled', or 'waiting'.")

In [5]:
class BookingSystem:
    def __init__(self):
        self._bookings = {}

    def book_conference(self, booking_id, conference_name, user_id):
        if booking_id in self._bookings:
            raise ValueError("Booking ID already exists.")
        new_booking = Booking(booking_id, conference_name, user_id)
        self._bookings[booking_id] = new_booking
        return new_booking

    def check_booking_status(self, booking_id):
        if booking_id not in self._bookings:
            raise ValueError("Booking ID does not exist.")
        booking = self._bookings[booking_id]
        return f"Booking ID: {booking.booking_id}, Status: {booking.status}, Conference: {booking.conference_name}, User ID: {booking.user_id}"
    
    def booking_exists(self, conference_name, user_id):
        for booking in self._bookings.values():
            if booking.conference_name == conference_name and booking.user_id == user_id:
                return True
        return False

    def get_booking_id(self, conference_name, user_id):
        for booking_id, booking in self._bookings.items():
            if booking.conference_name == conference_name and booking.user_id == user_id:
                return booking_id
        raise ValueError("No booking found for the given conference name and user ID.")
    
    def get_booking(self, conference_name, user_id):
        for booking_id, booking in self._bookings.items():
            if booking.conference_name == conference_name and booking.user_id == user_id:
                return booking
        raise ValueError("No booking found for the given conference name and user ID.")
    
    def cancel_booking(self, booking_id):
        if booking_id not in self._bookings:
            raise ValueError("Booking ID does not exist.")
        booking = self._bookings[booking_id]
        booking.status = "cancelled"
        # del self._bookings[booking_id]


# Conference

In [6]:
class Conference:
    def __init__(self, id, name, location, topics, start_timestamp, end_timestamp, available_slots):
        self._id = id
        self._name = name
        self._location = location
        self._topics = topics
        self._start_timestamp = start_timestamp
        self._end_timestamp = end_timestamp
        self._available_slots = available_slots
        self._waitlist = []

    # Getter and Setter for id
    @property
    def id(self):
        return self._id

    @id.setter
    def id(self, id):
        self._id = id

    # Getter and Setter for name
    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, name):
        self._name = name

    # Getter and Setter for location
    @property
    def location(self):
        return self._location

    @location.setter
    def location(self, location):
        self._location = location

    # Getter and Setter for topics
    @property
    def topics(self):
        return self._topics

    @topics.setter
    def topics(self, topics):
        self._topics = topics

    # Getter and Setter for start_timestamp
    @property
    def start_timestamp(self):
        return self._start_timestamp

    @start_timestamp.setter
    def start_timestamp(self, start_timestamp):
        self._start_timestamp = start_timestamp

    # Getter and Setter for end_timestamp
    @property
    def end_timestamp(self):
        return self._end_timestamp

    @end_timestamp.setter
    def end_timestamp(self, end_timestamp):
        self._end_timestamp = end_timestamp

    # Getter and Setter for available_slots
    @property
    def available_slots(self):
        return self._available_slots

    @available_slots.setter
    def available_slots(self, available_slots):
        self._available_slots = available_slots

    # Getter and Setter for waitlist
    @property
    def waitlist(self):
        return self._waitlist

    @waitlist.setter
    def waitlist(self, waitlist):
        self._waitlist = waitlist

    def add_to_waitlist(self, booking):
        if booking not in self._waitlist:
            self._waitlist.append(booking)
        else:
            print(f"{booking} is already on the waitlist.")

    def remove_from_waitlist(self, booking):
        if booking in self._waitlist:
            self._waitlist.remove(booking)
        else:
            print(f"{booking} is not on the waitlist.")

In [7]:
class ConferenceSystem:
    def __init__(self, user_system, booking_system):
        self.conferences = {}
        self._user_system = user_system
        self._booking_system = booking_system

    def add_conference(self, name, location, topics, start_timestamp, end_timestamp, available_slots):
        """
        Add Conference: Add a new conference. It takes following parameters as input
        Name: Any Alphanumeric String. Spaces are the only special character allowed
        Location: Any Alphanumeric String. Spaces are the only special characters allowed
        Topics: Comma Separated Strings. Strings can be Alphanumeric with spaces allowed. Maximum 10 Strings
        Timing
        Start Timestamp
        End Timestamp
        Conditions: Start Timestamp must be before End Timestamp. Duration should not exceed 12 hours
        Available Slots: A integer greater than 0
        Expectations: Conference Name should be unique globally unique in system. Trying to add a conference with the same name as existing one should result in error. There is no way to edit an added conference. The Timezone for Start and End Timestamp should be UTC.
        """

        # Validate name and location
        if not re.match(r'^[a-zA-Z0-9 ]+$', name):
            raise ValueError("Invalid conference name. Only alphanumeric characters and spaces are allowed.")
        if not re.match(r'^[a-zA-Z0-9 ]+$', location):
            raise ValueError("Invalid location. Only alphanumeric characters and spaces are allowed.")

        # Validate topics
        topics_list = [topic.strip() for topic in topics.split(",")]
        if len(topics_list) > 10 or any(not re.match(r'^[a-zA-Z0-9 ]+$', topic) for topic in topics_list):
            raise ValueError("Invalid topics. Must be comma-separated strings with alphanumeric characters and spaces, maximum 10 strings.")

        # Validate timestamps
        start_timestamp = datetime.strptime(start_timestamp, '%Y-%m-%d %H:%M:%S')
        end_timestamp = datetime.strptime(end_timestamp, '%Y-%m-%d %H:%M:%S')
        if start_timestamp >= end_timestamp:
            raise ValueError("Start timestamp must be before end timestamp.")
        if (end_timestamp - start_timestamp) > timedelta(hours=12):
            raise ValueError("Conference duration should not exceed 12 hours.")

        # Validate available slots
        if available_slots <= 0:
            raise ValueError("Available slots must be a positive integer.")

        # Check for unique name
        if name in self.conferences:
            raise ValueError("Conference name must be unique.")

        # Add the conference
        conference_id = str(uuid.uuid4())
        new_conference = Conference(conference_id, name, location, topics, start_timestamp, end_timestamp, available_slots)
        self.conferences[name] = new_conference

        return new_conference
    
    def conference_exists(self, name):
        """
        Check if a conference with the given name exists in the system.
        """
        return name in self.conferences
    
    def get_conference(self, name):
        """
        Return a conference.
        """
        return self.conferences[name]
    
    def _is_conference_started(self, conference):
        """
        Check if the conference has already started.
        """
        current_time = datetime.utcnow()
        return current_time >= conference.start_timestamp
    
    def book_conference(self, conference_name, user_id):
        if not self.conference_exists(conference_name):
            raise ValueError("Conference does not exist.")
        if not self._user_system.user_exists(user_id):
            raise ValueError("User does not exist.")
        if self._booking_system.booking_exists(conference_name, user_id):
            booking_id = self._booking_system.get_booking_id(conference_name, user_id)
            raise ValueError(f"User has already booked this conference with ID {booking_id}.")
        
        conference = self.get_conference(conference_name)
        if self._is_conference_started(conference):
            raise ValueError("Cannot book a conference that has already started.")
        
        user = self._user_system.get_user(user_id)
        if user.has_overlapping_conference(conference.start_timestamp, conference.end_timestamp, self.conferences):
            raise ValueError("User has overlapping conferences.")
        
        booking_id = str(uuid.uuid4())
        new_booking = self._booking_system.book_conference(booking_id, conference_name, user_id)
        user.add_booking(new_booking)
        
        if conference.available_slots > 0:
            conference.available_slots -= 1
            new_booking.status = 'accepted'
            return f"Booking successful. Your booking ID is {booking_id}."
        else:
            conference.add_to_waitlist(new_booking)
            return f"All slots are full. You have been added to the waitlist for conference '{conference_name}', with booking ID '{booking_id}'."

    def cancel_conference_booking(self, conference_name, user_id):
        if not self.conference_exists(conference_name):
            raise ValueError("Conference does not exist.")
        if not self._user_system.user_exists(user_id):
            raise ValueError("User does not exist.")
        if not self._booking_system.booking_exists(conference_name, user_id):
            raise ValueError("User does not have a booking for this conference.")
        
        conference = self.get_conference(conference_name)
        if self._is_conference_started(conference):
            raise ValueError("Cannot cancel a booking for a conference that has already started.")
        
        # Get the booking and conference details
        booking_id = self._booking_system.get_booking_id(conference_name, user_id)
        booking = self._booking_system.get_booking(conference_name, user_id)
        user = self._user_system.get_user(user_id)
        
        # Remove booking from the user
        user.remove_booking(booking_id)
        
        # Update conference availability
        if booking.status == 'accepted':
            conference.available_slots += 1
            if conference.waitlist:
                # Move the first person from the waitlist to a booked slot
                next_waitlist_booking = conference.waitlist.pop(0)
                next_user_id = next_waitlist_booking.user_id
                next_user = self._user_system.get_user(next_user_id)
                
                # Update waitlist booking status to accepted
                # user.accept_booking(booking)
                next_waitlist_booking.status = 'accepted'
                response = (f"Booking for conference '{conference_name}' has been canceled. "
                            f"User '{next_user_id}' has been moved from the waitlist to a booked slot.")
            else:
                response = f"Booking for conference '{conference_name}' has been canceled."
                
            # Remove the canceled booking from the booking system
            self._booking_system.cancel_booking(booking_id)
        else:
            # If booking was on the waitlist, just remove it from the waitlist
            conference.remove_from_waitlist(booking)
            self._booking_system.cancel_booking(booking_id)
            response = f"Booking for conference '{conference_name}' was on the waitlist and has been canceled."
        
        return response

In [8]:
user_system = UserSystem()
booking_system = BookingSystem()
conference_system = ConferenceSystem(user_system, booking_system)

In [9]:
try:
    new_user = user_system.add_user(
        user_id="User123",
        interested_topics="AI, Blockchain, Cybersecurity, Data Science"
    )
    print(f"User '{new_user.user_id}' added successfully!")
except ValueError as e:
    print(e)

User 'User123' added successfully!


In [10]:
try:
    new_user = user_system.add_user(
        user_id="User124",
        interested_topics="AI, Blockchain, Cybersecurity, Data Science"
    )
    print(f"User '{new_user.user_id}' added successfully!")
except ValueError as e:
    print(e)

User 'User124' added successfully!


In [17]:
try:
    new_conf = conference_system.add_conference(
        name="Tech Conference 2026",
        location="New York",
        topics="AI, Blockchain, Cybersecurity",
        start_timestamp="2024-08-03 09:00:00",
        end_timestamp="2024-09-03 17:00:00",
        available_slots=1
    )
    print(f"Conference '{new_conf.name}' added successfully!")
except ValueError as e:
    print(e)

Conference duration should not exceed 12 hours.


In [12]:
try:
    message = conference_system.book_conference("Tech Conference 2024", "User123")
    print(message)
except ValueError as e:
    print(e)

Cannot book a conference that has already started.


In [13]:
try:
    message = conference_system.book_conference("Tech Conference 2024", "User124")
    print(message)
except ValueError as e:
    print(e)

Cannot book a conference that has already started.


In [16]:
booking_system.check_booking_status("3cc2763e-9352-47cd-9501-83b2ac705644")

'Booking ID: 3cc2763e-9352-47cd-9501-83b2ac705644, Status: accepted, Conference: Tech Conference 2024, User ID: User123'

In [14]:
try:
    message = conference_system.cancel_conference_booking("Tech Conference 2024", "User123")
    print(message)
except ValueError as e:
    print(e)

User does not have a booking for this conference.
